Link da documentação https://platform.openai.com/docs/api-reference/introduction

Nos testes, não foi utilizado o pacote disponibilizado pela OpenAI para Python. Em vez disso, foi utilizada uma requisição via requests, pois essa abordagem pode ser aplicada em qualquer linguagem de programação.

Lista de comandos para linux ou mac
- Criar o anbiete virtual python -m venv nome_do_ambiente 
- Iniciar o anbiente python -m venv nome_do_ambiente (Para linux ou mac)
- Instalar os pacotes utilizado no abiente pip install -r requirements.tx




In [97]:
from dotenv import load_dotenv
import os
import requests
import json
import csv
import pandas as pd
from IPython.display import display
import json
import random
import faker


In [34]:
# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()

# Acessa a variável de ambiente API_KEY
API_KEY = os.getenv('API_KEY')

# Verifique se a API_KEY foi carregada corretamente
if not API_KEY:
    raise ValueError("API_KEY não foi carregada corretamente. Verifique o arquivo .env")



In [3]:

#Verificando os modelos disponibilizados para utilização
#API_KEY  esta variável tem o codígo da api da openai
# Define os headers e a URL da API
headers = {"Authorization": f"Bearer {API_KEY}"}
url = 'https://api.openai.com/v1/models'
# Faz a requisição GET para listar todos os modelos
response = requests.get(url, headers=headers)
# Verifica se a requisição foi bem-sucedida
if response.status_code == 200:
    models = response.json()

    # Especifica o nome do arquivo CSV
    csv_file = 'modelos_openai.csv'
    # Escreve os dados no arquivo CSV
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Escreve o cabeçalho
        writer.writerow(['Model ID', 'Object'])
        # Escreve os dados dos modelos
        for model in models['data']:
            writer.writerow([model['id'], model['object']])

    print(f"Lista de modelos salva no arquivo {csv_file}")
else:
    print(f"Falha ao recuperar modelos. Código de status: {response.status_code}")
    print(response.text)

Lista de modelos salva no arquivo modelos_openai.csv


In [4]:
#listando os modelos
df = pd.read_csv('modelos_openai.csv')
display(df)

,Model ID,Object
0,dall-e-3,model
1,gpt-4-1106-preview,model
2,whisper-1,model
3,gpt-4o-2024-05-13,model
4,babbage-002,model
5,dall-e-2,model
6,gpt-3.5-turbo-16k,model
7,tts-1-hd-1106,model
8,tts-1-hd,model
9,gpt-4o,model


In [37]:
def calculaNumeroTokens(nome_produto):
    text ="Pertence a qual categoria de produto?, e em qual subcategoria ele  se encaixa? Na sua resposta, coloque  apenas os nomes das categorias separadas por vírgula."
    content = f"O item {nome_produto}, {text}"
    # Dividir a string em tokens
    tokens = content.split()
    # Contar o número de tokens
    return len(tokens) 

In [38]:
numero_tokens  =  calculaNumeroTokens("Tinta")
print(f"Numero de tokens desta requisisão: {numero_tokens}")

Numero de tokens desta requisisão:28


In [88]:
def postChatGpt(content , api_key,  model = "gpt-3.5-turbo" ):
    headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
    link = 'https://api.openai.com/v1/chat/completions'
    mensagem = {
        "model": model,   
        "messages": [{"role": "user", 
                    "content":content
                    }]
    }
    #Ajustando para formato json para o post
    mensagem = json.dumps(mensagem)
    # Faz a requisição POST
    return requests.post(link, headers=headers, data=mensagem)

In [59]:

#content = "De agora em diante, você é um especialista em classificação e categorização de produtos. Vou fornecer algumas informações e quero que você vá categorizando à medida que conversamos."
#response= postChatGpt(content, API_KEY  )
#response

<Response [200]>

In [95]:
#Explorando a categorização sem presonalização 
#O sistema não mantem consistencia nas reponsta usando o mesmo titulo, isso pode ser um problema se tiver itens com mesmo titulo no arquivo de importação

nome_produto = "Tinta"  
content_1 = f"O item {nome_produto}, Pertence a qual categoria de produto.? Na sua resposta coloque somente o nome da categoria sem pontuação e a resposta deve ser em uma palavra ou uma palavra composta."
response_categoria_1 = postChatGpt(content_1, API_KEY  )
# Verifica se a requisição foi bem sucedida
if response_categoria_1.status_code == 200:
    # Extrai a resposta do chatbot
    response_categoria_1 = response_categoria_1.json()["choices"][0]["message"]["content"]
    print("Resposta categoria nível 1 :", response_categoria_1)
    
    ## Verificando o segundo nivel da categoria
    content_2 = f"O item {nome_produto} pertence a categoria {response_categoria_1}. Qual é a subcategoria deste produto? Na sua resposta, coloque somente o nome da subcategoria sem pontuação, em uma palavra ou uma palavra composta."
    response_categoria_2 = postChatGpt(content_2, API_KEY  )
    response_categoria_2 = response_categoria_2.json()["choices"][0]["message"]["content"]
    print("Resposta categoria nível 2:", response_categoria_2) 
    
    # Verificando a categoria nivel 3
    content_3 = f"O item {nome_produto} pertence a categoria {response_categoria_1} a sub categoria {response_categoria_2}. Qual é o terceiro nível de categoria deste produto? Na sua resposta, coloque apenas o nome da terceira categoria, sem pontuação, em uma palavra ou uma palavra composta."
    response_categoria_3 = postChatGpt(content_3, API_KEY  )
    response_categoria_3 = response_categoria_3.json()["choices"][0]["message"]["content"]
    print("Resposta categoria nível 3:", response_categoria_2)    
    
else:
    print("Erro ao realizar a requisição:", response.status_code, response.text)

# Imprime a resposta da requisição
# A responta fica em choices - message - content
# Note que na resposta o role tem o nome de 
#print(response)
#print(response.text)

Resposta categoria nível 1 : Materialconstrucao
Resposta categoria nível 2: Pintura
Resposta categoria nível 3: Pintura


In [103]:
#criando categoria fake  para treinar o modelo com dados personalizado

fake = faker.Faker()

produtos = []

# Conjuntos pré-definidos de categorias
categorias_nivel_1 = [
    "Eletrônicos",
    "Moda",
    "Casa e Escritório",
    "Esportes",
    "Livros",
    "Beleza",
    "Informática",
    "Automóvel",
    "Jogos",
    "Música"
]

categorias_nivel_2 = [
    "Telefones Celulares",
    "Tablets",
    "Notebooks",
    "Roupas",
    "Sapatos",
    "Móveis",
    "Decoração",
    "Futebol",
    "Tênis",
    "Basquete",
    "Romances",
    "Autoajuda",
    "Maquiagem",
    "Perfumes",
    "Componentes",
    "Acessórios",
    "Carros",
    "Motos",
    "Jogos de Tabuleiro",
    "Jogos de Console",
    "Instrumentos Musicais",
    "Discos"
]

categorias_nivel_3 = [
    "Smartphones",
    "iPhones",
    "Tablets Android",
    "iPads",
    "Notebooks Gamer",
    "Notebooks Ultraleves",
    "Camisetas",
    "Calças",
    "Vestidos",
    "Sapatos Casuais",
    "Sapatos de Salto",
    "Sofás",
    "Mesas",
    "Cadeiras",
    "Quadros",
    "Bolas de Futebol",
    "Chuteiras",
    "Raquetes de Tênis",
    "Tênis de Corrida",
    "Sapatilhas de Basquete",
    "Livros de Romance",
    "Livros de Autoajuda",
    "Maquiagem de Rosto",
    "Maquiagem de Olhos",
    "Perfumes Femininos",
    "Perfumes Masculinos",
    "Placas de Vídeo",
    "Processadores",
    "Memórias RAM",
    "Carros Sedan",
    "Carros SUV",
    "Motos Esportivas",
    "Motos de Passeio",
    "Jogos de Tabuleiro Clássicos",
    "Jogos de Tabuleiro Estratégicos",
    "Jogos de Console de Ação",
    "Jogos de Console de Esporte",
    "Violões",
    "Pianos",
    "Baterias",
    "Discos de Vinil",
    "Discos de Rock",
    "Discos de Pop"
]

for _ in range(500):
    titulo = fake.sentence(nb_words=4)

    # Selecionar aleatoriamente uma categoria de cada nível
    categoria_nivel_1 = random.choice(categorias_nivel_1)
    categoria_nivel_2 = random.choice(categorias_nivel_2)
    categoria_nivel_3 = random.choice(categorias_nivel_3)

    produto = {
        "titulo": titulo,
        "categorias": {
            "nivel_1": categoria_nivel_1,
            "nivel_2": categoria_nivel_2,
            "nivel_3": categoria_nivel_3
        }
    }

    produtos.append(produto)

with open("produtos.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)

    # Escrever o cabeçalho do CSV
    writer.writerow(["Título", "Categoria Nível 1", "Categoria Nível 2", "Categoria Nível 3"])
    # Escrever cada produto no CSV
    for produto in produtos:
        titulo = produto["titulo"]
        categoria_nivel_1 = produto["categorias"]["nivel_1"]
        categoria_nivel_2 = produto["categorias"]["nivel_2"]
        categoria_nivel_3 = produto["categorias"]["nivel_3"]

        writer.writerow([titulo, categoria_nivel_1, categoria_nivel_2, categoria_nivel_3])